In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import pandas as pd
books = pd.read_csv("../DATA/books_cleaned.csv")
books.head()

In [ ]:
books["tagged_description"].head()

In [ ]:
books["tagged_description"].to_csv("../DATA/tagged_description.txt",sep="\n",index=False,header=False)

In [ ]:
raw_documents = TextLoader("../DATA/tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0,chunk_overlap=0,separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [ ]:
documents[0]

In [ ]:
# Test OpenAI API connection
from openai import OpenAI
client = OpenAI()
try:
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input="Hello world"
    )
    print("API connection successful!")
except Exception as e:
    print(f"API connection failed: {e}")

In [ ]:
%pip install sentence_transformers

In [ ]:
# Use HuggingFace embeddings which run locally
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize the embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
db_books = Chroma.from_documents(documents, embedding=embeddings)

In [ ]:
# db_books = Chroma.from_documents(documents,embedding=OpenAIEmbeddings())



In [ ]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query,k=10)
docs

In [ ]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

In [ ]:
def retrieve_sementic_recommendations(query:str,top_k:int = 10) -> pd.DataFrame:
  recs = db_books.similarity_search(query,k=50)
  
  books_list = []
  
  for i in range(0,len(recs)):
    books_list += [int(recs[i].page_content.strip('"').split()[0])]
    
  return books[books["isbn13"].isin(books_list)]

In [ ]:
retrieve_sementic_recommendations("A book to teach children about nature")

In [ ]:
retrieve_sementic_recommendations("A book to teach me how to build an AI model").sample(7)